In [1]:
import pandas as pd
import numpy as np
import os

### 2018年6月 - 2021年6月基金日净值数据

In [5]:
# 按文件夹搜索导入数据
mydatalist = []
for info in os.listdir('2.基金数据210630/180630/'):
    domain = os.path.abspath('2.基金数据210630/180630/')
    info = os.path.join(domain,info)
    data = pd.read_csv(info)
    mydatalist.append(data)

nav_data = pd.concat(mydatalist)

#### 基金主体信息合并

In [8]:
maininfo = pd.read_csv('2.基金数据210630/FUND_MainInfo.csv') # 仅保留了契约开放式的股票型&混合型基金

In [9]:
maininfo.info(1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2404 entries, 0 to 2403
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   FundID           2404 non-null   int64  
 1   MasterFundCode   2404 non-null   int64  
 2   FullName         2404 non-null   object 
 3   FundCompanyID    2404 non-null   int64  
 4   FundCompanyName  2404 non-null   object 
 5   ManagementFee    2404 non-null   float64
 6   InceptionDate    2404 non-null   object 
 7   FundTypeID       2404 non-null   object 
 8   FundType         2404 non-null   object 
 9   CategoryID       2404 non-null   object 
 10  Category         2404 non-null   object 
dtypes: float64(1), int64(3), object(7)
memory usage: 206.7+ KB


In [10]:
unitinfo = pd.read_csv('2.基金数据210630/FUND_UnitClassInfo.csv') # 仅保留了所有非定期开放式基金

In [11]:
unitinfo.info(1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13039 entries, 0 to 13038
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   FundID           13039 non-null  int64  
 1   MasterFundCode   12991 non-null  float64
 2   FundClassID      13039 non-null  float64
 3   Symbol           13039 non-null  int64  
 4   Expired          13039 non-null  int64  
 5   ShortName        13039 non-null  object 
 6   InceptionDate    13039 non-null  object 
 7   Cycle            0 non-null      float64
 8   RedeemStartDate  11617 non-null  object 
dtypes: float64(3), int64(3), object(3)
memory usage: 916.9+ KB


In [12]:
print(maininfo.shape, unitinfo.shape)

(2404, 11) (13039, 9)


In [13]:
print(len(maininfo.FundID.unique()),len(maininfo.MasterFundCode.unique()))

2404 2404


In [14]:
print(len(unitinfo.FundID.unique()),len(unitinfo.MasterFundCode.unique()))

7929 7929


In [15]:
fundsinfo = pd.merge(maininfo,unitinfo, how = 'inner', on = 'FundID')
print(fundsinfo.shape)

(2254, 19)


In [16]:
fundsinfo.head(1)
fundsinfo = fundsinfo.drop(columns =['MasterFundCode_x','MasterFundCode_y','InceptionDate_x','Expired','Cycle'])

In [17]:
print(len(fundsinfo.Symbol.unique()),len(fundsinfo.FundClassID.unique()))

2237 2237


#### 基金日净值与主体信息合并

对比合并整理后的基金主体信息与日净值表格中的索引列’基金代码‘、’基金份额类别ID’，发现单个基金代码及份额类别ID存在对应多支基金主体信息的情况，但这类索引列在日净值表上对应的净值为单日单个，意味着不同主体信息对应同一索引ID的多支基金存在主要基金与连接基金的关系，其净值数据实为一支基金的净值，对于这类重复索引值的基金，作仅保留起始日期较早一支的处理。

In [18]:
fundsinfo1 = fundsinfo[['FundID','FundCompanyID','ManagementFee',\
                        'Symbol','ShortName','InceptionDate_y',\
                        'RedeemStartDate']].drop_duplicates(subset=['Symbol'], keep='first') # 按索引值基金代码去重

In [19]:
nav_data1 = pd.merge(fundsinfo1, nav_data, how='inner', on='Symbol')
print(nav_data1.shape)

(1259535, 11)


In [20]:
# 将可赎回起始日中空值用其份额成立日补齐
nav_data1.loc[nav_data1['RedeemStartDate'].isnull(),'RedeemStartDate']= nav_data1[nav_data1['RedeemStartDate'].isnull()]['InceptionDate_y']
# Check Output
len(nav_data1[nav_data1.RedeemStartDate.isna()])

0

In [21]:
dt = nav_data1.drop(columns=['Symbol','InceptionDate_y','FundClassID']) # 去掉多余列

In [22]:
dt.head(5)

,FundID,FundCompanyID,ManagementFee,ShortName,RedeemStartDate,TradingDate,NAV,AccumulativeNAV
0,108426,1059,1.5,华夏成长混合,2002/1/30,2018/6/30,1.045,3.456
1,108426,1059,1.5,华夏成长混合,2002/1/30,2018/7/2,1.032,3.443
2,108426,1059,1.5,华夏成长混合,2002/1/30,2018/7/3,1.023,3.434
3,108426,1059,1.5,华夏成长混合,2002/1/30,2018/7/4,1.011,3.422
4,108426,1059,1.5,华夏成长混合,2002/1/30,2018/7/5,0.995,3.406


In [25]:
# dt.to_csv('data1806_2106.csv',encoding='utf-8-sig') #导出一次数据

In [47]:
# 调整时间变量
dt['RedeemStartDate'] = pd.to_datetime(dt['RedeemStartDate'], format='%Y-%m-%d')
dt['TradingDate'] = pd.to_datetime(dt['TradingDate'], format='%Y-%m-%d')

In [52]:
# 计算基金回报率和净回报率（年化）
dt['acc_nav(初)'] = dt.groupby('FundID')['AccumulativeNAV'].shift(1)
dt['ri'] = (dt['AccumulativeNAV']/dt['acc_nav(初)']-1)*360 # 年化总回报率
dt['net_ri'] = dt['ri'] - dt['ManagementFee'] # 年化净回报率

In [54]:
dt0 = dt[['FundID','FundCompanyID','TradingDate','ri','net_ri']]

In [55]:
dt0.head(5)

,FundID,FundCompanyID,TradingDate,ri,net_ri
0,108426,1059,2018-06-30,NaN,NaN
1,108426,1059,2018-07-02,-1.354167,-2.854167
2,108426,1059,2018-07-03,-0.941040,-2.441040
3,108426,1059,2018-07-04,-1.258008,-2.758008
4,108426,1059,2018-07-05,-1.683226,-3.183226


In [66]:
columns = ['fund_id','compy_id','trddy','ri','net_ri']
dt0.columns = columns

### Cahart数据整理

1. 提取所需因子，加入债券市场因子
2. 选取一定时间内数据

In [35]:
fivefactor = pd.read_csv('2.基金数据210630/fivefactor_daily.csv')
rbrf = pd.read_csv('2.基金数据210630/RBRF_daliy.csv') # 导入中债国债收益率

In [32]:
print(fivefactor.head(5), fivefactor.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6667 entries, 0 to 6666
Data columns (total 13 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   trddy      6667 non-null   object 
 1   mkt_rf     6667 non-null   float64
 2   smb        6667 non-null   float64
 3   hml        6667 non-null   float64
 4   umd        6667 non-null   float64
 5   rmw        6667 non-null   float64
 6   cma        6667 non-null   float64
 7   rf         6667 non-null   float64
 8   smb_equal  6667 non-null   float64
 9   hml_equal  6667 non-null   float64
 10  umd_equal  6667 non-null   float64
 11  rmw_equal  6667 non-null   float64
 12  cma_equal  6667 non-null   float64
dtypes: float64(12), object(1)
memory usage: 677.2+ KB
        trddy    mkt_rf       smb       hml       umd       rmw       cma  \
0  1994-01-04 -0.003950  0.000329  0.003827  0.010374 -0.004755 -0.014706   
1  1994-01-05  0.007166  0.011850  0.012679  0.009254 -0.020330 -0.006645   
2  1994

In [33]:
print(rbrf.head(5), rbrf.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4867 entries, 0 to 4866
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Trddt   4867 non-null   object 
 1   Yield   4867 non-null   float64
dtypes: float64(1), object(1)
memory usage: 76.2+ KB
       Trddt   Yield
0   2002/1/4  2.5132
1   2002/1/7  2.5315
2   2002/1/8  1.6574
3   2002/1/9  1.6275
4  2002/1/10  1.6239 None


In [36]:
# 调整时间列格式
fivefactor['trddy'] = pd.to_datetime(fivefactor['trddy'], format='%Y-%m-%d')
rbrf['Trddt'] = pd.to_datetime(rbrf['Trddt'], format='%Y-%m-%d')

In [37]:
f = pd.merge(fivefactor, rbrf, how='inner', left_on = 'trddy', right_on = 'Trddt')

In [39]:
f['rbrf'] = f['Yield'] - f['rf']

In [42]:
factor = f[['trddy','mkt_rf','smb','hml','umd','rbrf','rf']]

In [56]:
factor.head(5)

,trddy,mkt_rf,smb,hml,umd,rbrf,rf
0,2002-01-04,-0.019175,-0.000587,-0.004185,0.001121,2.513139,0.000061
1,2002-01-07,-0.011709,0.000513,-0.001980,0.003688,2.531439,0.000061
2,2002-01-08,-0.008220,-0.001604,-0.001937,0.002960,1.657339,0.000061
3,2002-01-09,-0.018483,-0.004634,-0.002118,0.007703,1.627439,0.000061
4,2002-01-10,0.008741,0.003714,0.007480,-0.004754,1.623839,0.000061


### 基金回报率数据与因子数据合并

In [68]:
# 取2018-7-2到2021-6-4内数据
dt1 = dt0[(dt0['trddy']>='2018-07-02')&(dt0['trddy']<='2021-06-04')]

In [84]:
m_dt = pd.merge(dt1, factor, how='left', on='trddy') # 与factor因子合并
m_dt1 = m_dt.fillna(method='ffill') # 填补缺失值

In [85]:
# 导出数据
m_dt1.to_csv('model_data180630.csv', encoding = 'utf-8-sig')